In [ ]:
import os
os.environ['GOOGLE_API_KEY'] = 'Key'

In [2]:
!pip install -q langchain-google-genai langchain-core requests

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 444.0/444.0 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 29.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-generativeai 0.8.5 requires google-ai-generativelanguage==0.6.15, but you have google-ai-generativelanguage 0.6.18 which is incompatible.


In [3]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings, ChatGoogleGenerativeAI
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage
import requests

In [4]:
# tool create

@tool
def multiply(a: int, b: int) -> int:
  """Given 2 numbers a and b this tool returns their product"""
  return a * b

In [5]:
print(multiply.invoke({'a':3, 'b':4}))

12


In [6]:
multiply.name

'multiply'

In [7]:
multiply.description

'Given 2 numbers a and b this tool returns their product'

In [8]:
multiply.args

{'a': {'title': 'A', 'type': 'integer'},
 'b': {'title': 'B', 'type': 'integer'}}

In [9]:
# tool binding

In [11]:
llm = ChatGoogleGenerativeAI(model='gemini-2.5-flash')

In [12]:
llm.invoke('hi')

AIMessage(content='Hi there! How can I help you today?', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': []}, id='run--868bbea6-8f07-406b-8a8a-10490d3a8a2d-0', usage_metadata={'input_tokens': 2, 'output_tokens': 35, 'total_tokens': 37, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 25}})

In [14]:
llm_with_tools = llm.bind_tools([multiply])

In [15]:
llm_with_tools.invoke('Hi how are you')

AIMessage(content="I'm doing well, thank you! How can I help you today?", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': []}, id='run--e0f23354-9a3e-4f17-aec0-71b6ca6de685-0', usage_metadata={'input_tokens': 58, 'output_tokens': 16, 'total_tokens': 74, 'input_token_details': {'cache_read': 0}})

In [16]:
query = HumanMessage('can you multiply 3 with 1000')

In [17]:
messages = [query]

In [18]:
messages

[HumanMessage(content='can you multiply 3 with 1000', additional_kwargs={}, response_metadata={})]

In [19]:
result = llm_with_tools.invoke(messages)

In [20]:
messages.append(result)

In [21]:
messages

[HumanMessage(content='can you multiply 3 with 1000', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'function_call': {'name': 'multiply', 'arguments': '{"a": 3.0, "b": 1000.0}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': []}, id='run--3e6ef3ae-0a72-4b8b-b5cc-03d01e5494e5-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3.0, 'b': 1000.0}, 'id': '37b0cdfd-473b-4d01-9a53-cb1c8e81a82c', 'type': 'tool_call'}], usage_metadata={'input_tokens': 65, 'output_tokens': 98, 'total_tokens': 163, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 77}})]

In [22]:
tool_result = multiply.invoke(result.tool_calls[0])

In [23]:
tool_result

ToolMessage(content='3000', name='multiply', tool_call_id='37b0cdfd-473b-4d01-9a53-cb1c8e81a82c')

In [24]:
messages.append(tool_result)

In [25]:
messages

[HumanMessage(content='can you multiply 3 with 1000', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'function_call': {'name': 'multiply', 'arguments': '{"a": 3.0, "b": 1000.0}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': []}, id='run--3e6ef3ae-0a72-4b8b-b5cc-03d01e5494e5-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3.0, 'b': 1000.0}, 'id': '37b0cdfd-473b-4d01-9a53-cb1c8e81a82c', 'type': 'tool_call'}], usage_metadata={'input_tokens': 65, 'output_tokens': 98, 'total_tokens': 163, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 77}}),
 ToolMessage(content='3000', name='multiply', tool_call_id='37b0cdfd-473b-4d01-9a53-cb1c8e81a82c')]

In [26]:
llm_with_tools.invoke(messages).content

'The product of 3 and 1000 is 3000.'

##Calling the currency exchange agent that calls the tool ( fetches the exchange factor from the URL real time)

In [27]:
# tool create
from langchain_core.tools import InjectedToolArg
from typing import Annotated

@tool
def get_conversion_factor(base_currency: str, target_currency: str) -> float:
  """
  This function fetches the currency conversion factor between a given base currency and a target currency
  """
  url = f'https://v6.exchangerate-api.com/v6/b995edb4b1506ad83a9137aa/pair/{base_currency}/{target_currency}'

  response = requests.get(url)

  return response.json()

@tool
def convert(base_currency_value: int, conversion_rate: Annotated[float, InjectedToolArg]) -> float:
  """
  This function converts the base currency to the target currency.

  """

  return base_currency_value * conversion_rate


In [28]:
convert.args

{'base_currency_value': {'title': 'Base Currency Value', 'type': 'integer'},
 'conversion_rate': {'title': 'Conversion Rate', 'type': 'number'}}

In [29]:
get_conversion_factor.invoke({'base_currency':'USD','target_currency':'NPR'})

{'result': 'success',
 'documentation': 'https://www.exchangerate-api.com/docs',
 'terms_of_use': 'https://www.exchangerate-api.com/terms',
 'time_last_update_unix': 1756771201,
 'time_last_update_utc': 'Tue, 02 Sep 2025 00:00:01 +0000',
 'time_next_update_unix': 1756857601,
 'time_next_update_utc': 'Wed, 03 Sep 2025 00:00:01 +0000',
 'base_code': 'USD',
 'target_code': 'NPR',
 'conversion_rate': 141.0276}

In [30]:
convert.invoke({'base_currency_value':10, 'conversion_rate':140.23})

1402.3

In [31]:
# tool binding
llm = ChatGoogleGenerativeAI(model='gemini-2.5-flash')

In [32]:
llm_with_tools = llm.bind_tools([get_conversion_factor, convert])

In [33]:
messages = [HumanMessage('What is the conversion factor between NPR and USD, and based on that can you convert 10 npr to usd')]

In [34]:
messages

[HumanMessage(content='What is the conversion factor between NPR and USD, and based on that can you convert 10 npr to usd', additional_kwargs={}, response_metadata={})]

In [35]:
ai_message = llm_with_tools.invoke(messages)

In [37]:
messages.append(ai_message)

In [38]:
messages

[HumanMessage(content='What is the conversion factor between NPR and USD, and based on that can you convert 10 npr to usd', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'function_call': {'name': 'get_conversion_factor', 'arguments': '{"target_currency": "USD", "base_currency": "NPR"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': []}, id='run--f0f611b2-7f9e-4a7d-8e25-a11175b7e7d9-0', tool_calls=[{'name': 'get_conversion_factor', 'args': {'target_currency': 'USD', 'base_currency': 'NPR'}, 'id': '8731606c-d550-41fd-ae2a-7f8136ea8dab', 'type': 'tool_call'}], usage_metadata={'input_tokens': 143, 'output_tokens': 654, 'total_tokens': 797, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 627}})]

In [39]:
ai_message.tool_calls

[{'name': 'get_conversion_factor',
  'args': {'target_currency': 'USD', 'base_currency': 'NPR'},
  'id': '8731606c-d550-41fd-ae2a-7f8136ea8dab',
  'type': 'tool_call'}]

In [40]:
import json

for tool_call in ai_message.tool_calls:
  # execute the 1st tool and get the value of conversion rate
  if tool_call['name'] == 'get_conversion_factor':
    tool_message1 = get_conversion_factor.invoke(tool_call)
    # fetch this conversion rate
    conversion_rate = json.loads(tool_message1.content)['conversion_rate']
    # append this tool message to messages list
    messages.append(tool_message1)
  # execute the 2nd tool using the conversion rate from tool 1
  if tool_call['name'] == 'convert':
    # fetch the current arg
    tool_call['args']['conversion_rate'] = conversion_rate
    tool_message2 = convert.invoke(tool_call)
    messages.append(tool_message2)



In [42]:
llm_with_tools.invoke(messages).content

'The conversion factor between NPR and USD is 0.007091.\n10 NPR is equal to 0.07091 USD.'